In [1]:
print('ok')

ok


In [2]:
%pwd

'd:\\DataScience_Projects\\OpenAI\\End_to_End_MedicalChatBot_Llama\\End_to_End_MedicalChatBot_Llama2\\research'

In [3]:
import os
os.chdir("../")# 

In [4]:
%pwd

'd:\\DataScience_Projects\\OpenAI\\End_to_End_MedicalChatBot_Llama\\End_to_End_MedicalChatBot_Llama2'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='data/')

In [15]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
# text_chunks

In [11]:
# !pip uninstall -y sentence-transformers
# !pip install sentence-transformers
##### !pip install sentence-transformers==2.2.2

from tqdm.autonotebook import tqdm, trange
import sentence_transformers
print(sentence_transformers.__version__)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2504\1384648026.py:5: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


3.2.1


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2504\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\ASUS\anaconda3\envs\MChatBot_2\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by s

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [24]:
# query_result

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalchatbot4"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '18e4306999dbdf0bafecb91a71fc870e', 'Date': 'Mon, 04 Nov 2024 05:27:08 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [22]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch

In [24]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [25]:
retrieved_docs = retriever.invoke("What is stress?")


In [26]:

retrieved_docs

[Document(id='d0648734-edfa-44c7-9380-ec7788c5efa6', metadata={'page': 330.0, 'source': 'data\\Medical_book.pdf'}, page_content='es to stress, or by certain disease processes or medications.\nGALE ENCYCLOPEDIA OF MEDICINE 2 317\nAnxiety\nGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 317'),
 Document(id='f407b147-43f3-49bf-8353-3614e4caada4', metadata={'page': 330.0, 'source': 'data\\Medical_book.pdf'}, page_content='es to stress, or by certain disease processes or medications.\nGALE ENCYCLOPEDIA OF MEDICINE 2 317\nAnxiety\nGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 317'),
 Document(id='94cf5973-3507-4f3f-b6fb-fd146ba11c5a', metadata={'page': 330.0, 'source': 'data\\Medical_book.pdf'}, page_content='es to stress, or by certain disease processes or medications.\nGALE ENCYCLOPEDIA OF MEDICINE 2 317\nAnxiety\nGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 317')]

In [27]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [29]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({"input": "what is stress?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

In [ ]:
#  USING GOOGLE API

#  USING GOOGLE API

In [62]:
# import google.generativeai as genai
# import os
# from langchain.document_loaders import PyPDFLoader, DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
# from dotenv import load_dotenv




In [63]:
# # Load environment variables
# load_dotenv()
# PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
# GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [64]:
# # Configure Google API key
# genai.configure(api_key=GOOGLE_API_KEY)

In [65]:
# # Step 1: Load PDF Data
# def load_pdf_file(data):
#     loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
#     documents = loader.load()
#     return documents

In [66]:
# extracted_data = load_pdf_file(data='data/')

In [67]:
# # Step 2: Split Data into Chunks
# def text_split(extracted_data):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
#     text_chunks = text_splitter.split_documents(extracted_data)
#     return text_chunks

In [68]:
# text_chunks = text_split(extracted_data)

In [70]:
# # Step 5: Set Up Prompt and Question-Answering with Google PaLM
# from langchain.prompts import PromptTemplate

In [71]:
# # Define prompt template
# system_prompt = (
#     "You are an assistant for medical question-answering tasks. "
#     "Use the retrieved context below to answer the question concisely. "
#     "If unsure, state 'I don't know'. Limit response to three sentences."
#     "\n\n"
#     "{context}"
# )

In [85]:
# # # # Step 6: Function to Retrieve and Generate Answer using Google PaLM
# # # def query_google_palm(question, docsearch):
# # #     # Retrieve documents based on question
# # #     retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# # #     retrieved_docs = retriever.invoke(question)
# # #     context = "\n".join([doc.page_content for doc in retrieved_docs])

# # #     # Generate answer with Google PaLM
# # #     response = genai.generate_text(
# # #         prompt=system_prompt.format(context=context),
# # #         input=question,
# # #         max_output_tokens=200,
# # #         temperature=0.2
# # #     )
# # #     return response.generations[0].text.strip()


# # # Step 6: Function to Retrieve and Generate Answer using Google PaLM
# # def query_google_palm(question, docsearch):
# #     # Retrieve documents based on question
# #     retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# #     retrieved_docs = retriever.invoke(question)
# #     context = "\n".join([doc.page_content for doc in retrieved_docs])

# #     # Generate answer with Google PaLM using `generate` instead of `generate_text`
# #     response = genai.generate(
# #         model="text-bison-001",  # Specify the model name explicitly
# #         prompt=system_prompt.format(context=context),
# #         max_output_tokens=200,
# #         temperature=0.2
# #     )
# #     return response.generations[0]["content"].strip()

# # # Example Query
# # response = query_google_palm("What is Acne?", docsearch)
# # print("Response:", response)


# # Step 6: Function to Retrieve and Generate Answer using Google PaLM
# def query_google_palm(question, docsearch):
#     # Retrieve documents based on question
#     retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
#     retrieved_docs = retriever.invoke(question)
#     context = "\n".join([doc.page_content for doc in retrieved_docs])

#     # Generate answer with Google PaLM using `chat`
#     response = genai.chat(
#         model="chat-bison-001",  # Specify the appropriate chat model name
#         messages=[
#             {"role": "system", "content": system_prompt.format(context=context)},
#             {"role": "user", "content": question}
#         ],
#         max_output_tokens=200,
#         temperature=0.2
#     )
#     return response["candidates"][0]["content"].strip()

# # Example Query
# response = query_google_palm("What is Acne?", docsearch)
# print("Response:", response)


# # Simplified function
# def query_google_palm(question, docsearch):
#     retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
#     retrieved_docs = retriever.invoke(question)
#     context = "\n".join([doc.page_content for doc in retrieved_docs])

#     # Generate text with Google PaLM
#     response = genai.generate_text(
#         prompt=f"{system_prompt.format(context=context)}\n\n{question}",
#         model="text-bison-001",  # Replace with appropriate model if necessary
#         max_output_tokens=200,
#         temperature=0.2
#     )
#     return response.result.strip()

# # Example query
# response = query_google_palm("What is Acne?", docsearch)
# print("Response:", response)

# # Initialize and configure the generative model
# model = genai.GenerativeModel(model_name="text-bison-001")

# # Function to retrieve and generate an answer
# def query_google_palm(question, docsearch):
#     # Retrieve documents based on the question
#     retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
#     retrieved_docs = retriever.invoke(question)
#     context = "\n".join([doc.page_content for doc in retrieved_docs])

#     # Use model to generate an answer
#     input_prompt = f"{system_prompt.format(context=context)}\n\n{question}"
#     response = model.generate_content(prompt=input_prompt, max_output_tokens=200, temperature=0.2)

#     return response.generations[0].text.strip() if response.generations else "No response generated."

# # Example Query
# response = query_google_palm("What is Acne?", docsearch)
# print("Response:", response)

In [76]:
# # Example Query
# response = query_google_palm("What is Acne?", docsearch)
# print("Response:", response)

In [82]:
# print(dir(genai))

['ChatSession', 'GenerationConfig', 'GenerativeModel', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'annotations', 'caching', 'configure', 'create_tuned_model', 'delete_file', 'delete_tuned_model', 'embed_content', 'embed_content_async', 'get_base_model', 'get_file', 'get_model', 'get_operation', 'get_tuned_model', 'list_files', 'list_models', 'list_operations', 'list_tuned_models', 'protos', 'responder', 'string_utils', 'types', 'update_tuned_model', 'upload_file', 'utils']


In [ ]:
# LLAMA 2 model

# LLAMA 2 model

In [40]:
# # !pip install ctransformers
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

In [36]:
# # Step 5: Set Up Prompt and Question-Answering with Google PaLM
from langchain.prompts import PromptTemplate

In [44]:
# LLAMA 2 model


prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [38]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [41]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [42]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [43]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2504\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Stress is a psychological and physiological response to a perceived threat or challenge, which can be caused by a variety of factors such as work pressures, financial problems, relationship issues, or physical and emotional exhaustion. It can also be triggered by certain disease processes or medications.


KeyboardInterrupt: 